In [50]:
%matplotlib inline

import numpy as np
import re
import pickle
import pandas as pd
import matplotlib.pyplot as pl

In [65]:
cols = ['Project Number', 'Institution', 'University', 'Approved Amount']
na_values = ['data not included in P3', 'Nicht zuteilbar - NA']
    
dtypes = {
        'Approved Amount': np.float64
}

raw = pd.read_csv('P3_GrantExport.csv', sep = ';', index_col='Project Number', dtype=dtypes, usecols=cols, na_values=na_values)
df = raw.dropna()

In [174]:
df.sample(10)

,Institution,University,Approved Amount
Project Number,,,
154521,Forschungsstelle für Sozial- und Wirtschaftsge...,Universität Zürich - ZH,5900.0
163040,Département des neurosciences fondamentales Fa...,Université de Genève - GE,812244.0
52870,Deutsches Seminar Universität Basel,Universität Basel - BS,105742.0
61817,Cinémathèque suisse Centre d'Archivage,"NPO (Biblioth., Museen, Verwalt.) - NPO",439641.0
56640,Geologisch-Paläontologisches Institut Universi...,Universität Basel - BS,133484.0
27714,Département d'Ecologie et d'Evolution Faculté ...,Université de Lausanne - LA,138540.0
5813,Institut für Pflanzenwissenschaften Universitä...,Universität Bern - BE,51500.0
113524,Unité d'Histologie Département de Médecine Uni...,Université de Fribourg - FR,131300.0
157122,Eidg. Materialprüfungs- und Forschungsanstalt ...,Eidg. Material und Prüfungsanstalt - EMPA,254491.0


In [67]:
df.index.is_unique

True

In [177]:
with_canton = df.copy()

def ex_canton(text, axis):
    res = text.split('-')
    if len(res) < 2:
        return text
    else:
        return res[1]
    
def ex_uni(text, axis):
    res = text.split('-')
    if len(res) < 2:
        return text.strip()
    else:
        return res[0].strip()

with_canton['Canton'] = with_canton['University'].apply(ex_canton, axis=1)
with_canton['University'] = with_canton['University'].apply(ex_uni, axis=1)

with_canton.sample(10)

,Institution,University,Approved Amount,Canton
Project Number,,,,
117996,Institut für Geologie Universität Bern,Universität Bern,402651.0,BE
103991,Lab. de mesure et d'analyse des mouvements EPF...,EPF Lausanne,100423.0,EPFL
132509,Paul Scherrer Institut,Paul Scherrer Institut,177448.0,PSI
119643,Hochschule für Soziale Arbeit Fachhochschule N...,Fachhochschule Nordwestschweiz (ohne PH),5030.0,FHNW
26538,Centre Universitaire d'Informatique Université...,Université de Genève,124991.0,GE
25317,Département de Chimie Physique Faculté des Sci...,Université de Genève,158895.0,GE
126085,Empa Laboratory for Advanced Fibers,Eidg. Material und Prüfungsanstalt,340889.0,EMPA
35682,Département de Psychologie Université de Fribourg,Université de Fribourg,623628.0,FR
157848,Département d'histoire générale Faculté des le...,Université de Genève,11350.0,GE


In [178]:
grouped = with_canton.groupby(['Canton', 'University']).sum().reset_index()
len(grouped)

76

In [179]:
cantons = pd.read_csv('cantons.csv', header=None)
cantons = cantons[0].values.tolist()

print(cantons)

['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO', 'BS', 'BL', 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD', 'VS', 'NE', 'GE', 'JU', 'CH']


In [182]:
def is_known_canton(x, axis):
    return x.strip() in cantons

wc = grouped.copy()
wc['IsCanton'] = wc['Canton'].apply(is_known_canton, axis=1)

In [183]:
len(wc[wc['IsCanton'] == False])

68

In [184]:
wc[wc['IsCanton'] == False]

,Canton,University,Approved Amount,IsCanton
0,AGS,Forschungsanstalten Agroscope,3.311572e+07,False
1,AORI,AO Research Institute,3.435621e+06,False
2,ASP,Pädag. Hochschule Tessin (Teilschule SUPSI),1.593170e+05,False
3,ASPIT,Weitere Spitäler,1.074981e+07,False
5,BFH,Berner Fachhochschule,3.102870e+07,False
6,BITG,Biotechnologie Institut Thurgau,2.492535e+06,False
8,CREALP,Centre de rech. sur l'environnement alpin,1.567678e+06,False
9,CSEM,Swiss Center for Electronics and Microtech.,1.806825e+07,False
10,EAWAG,Eidg. Anstalt für Wasserversorgung,7.397585e+07,False
11,EHB,Eidg. Hochschulinstitut für Berufsbildung,2.086572e+06,False
